In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import math 

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

# Gráficos
# ==============================================================================
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Asunciones y Preprocesamiento
# ==============================================================================
from scipy import stats
import math
from scipy.stats import levene
import researchpy as rp
from sklearn.preprocessing import StandardScaler
import itertools

# ANOVA
# ==============================================================================
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA

#Linear Regresion con Sklearn
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Tamaño gráficas
# ==============================================================================
plt.rcParams["figure.figsize"] = (10,8)

from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler,OneHotEncoder

In [2]:
df= pd.read_csv("Bicicletas_limpio.csv", index_col = 0)
df

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,726,27-12-2019,spring,1,12,0,4,1,2,10.420847,11.33210,65.2917,23.458911,247,1867,2114
726,727,28-12-2019,spring,1,12,0,5,1,2,10.386653,12.75230,59.0000,10.416557,644,2451,3095
727,728,29-12-2019,spring,1,12,0,6,0,2,10.386653,12.12000,75.2917,8.333661,159,1182,1341
728,729,30-12-2019,spring,1,12,0,0,0,1,10.489153,11.58500,48.3333,23.500518,364,1432,1796


### Hacemos el modelo sin reemplazar outliers y sin categorizar 

In [3]:
df = df.drop(columns = ['instant', 'dteday'])

In [4]:
df.dtypes

season         object
yr              int64
mnth            int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

- No incluimos la variable categórica 'season'

In [5]:
numericas = df.select_dtypes(include=np.number)

In [6]:
x = numericas.drop('registered', axis=1) #Eliminamos nuestra variable respuesta.
y = numericas[['registered']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_test =lr.predict(x_test)
y_pred_train =lr.predict(x_train)

def metricas(y_test, y_train, y_test_pred, y_train_pred):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    return df

results = metricas(y_test, y_train,y_pred_test, y_pred_train)
results

,MAE,MSE,RMSE,R2,set
0,7.817917e-13,9.978574e-25,9.989281e-13,1.0,test
1,7.745037e-13,1.009388e-24,1.004683e-12,1.0,train


Vemos como queda sin variable categórica y sin otras variables que no tienen sentido compararlas con nuestra variable registrados.

In [11]:
x = numericas.drop(columns = ['registered', 'casual', 'cnt']) 
y = numericas[['registered']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_test =lr.predict(x_test)
y_pred_train =lr.predict(x_train)

def metricas(y_test, y_train, y_test_pred, y_train_pred):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    return df

results = metricas(y_test, y_train,y_pred_test, y_pred_train)
results

,MAE,MSE,RMSE,R2,set
0,490.892424,394844.970673,628.366908,0.817080,test
1,544.373922,544852.641399,738.141342,0.781454,train


    Obtenemos mejo R2 sin la variable categóricas pero eliminando otras variables. Pero las métricas son bastantes malas.

- Incluimos la variable categórica codificandola con un mapa

In [12]:
df_mapa = df.copy()

In [13]:
mapa_season = {"spring": 0,"summer": 1, "autumn": 2, "winter": 3} #asigno a cada estacion un valor
df_mapa["season"] = df_mapa["season"].map(mapa_season)#asigno a la columna los valores

In [20]:
x = df_mapa.drop(columns = ['registered', 'casual', 'cnt']) 
y = df_dummies[['registered']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_test =lr.predict(x_test)
y_pred_train =lr.predict(x_train)

def metricas(y_test, y_train, y_test_pred, y_train_pred):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    return df

results = metricas(y_test, y_train,y_pred_test, y_pred_train)
results

,MAE,MSE,RMSE,R2,set
0,447.537187,307374.825631,554.413948,0.857603,test
1,512.434138,479414.893527,692.397930,0.807702,train


     Hemos obtenido con la variable categórica peor R2 en el cojunto de entrenamiento pero mejor en el conjunto de test. Aunque las demás métricas son bastantes malas, veremos si podemos mejorarlo

In [16]:
df_dummies = df.copy()

In [17]:
dummies_season = pd.get_dummies(df_dummies['season'], prefix_sep= '_', prefix='season', dtype='int')

In [18]:
df_dummies[dummies_season.columns] = dummies_season

In [19]:
df_dummies.drop(['season'], axis=1, inplace=True)

In [21]:
x = df_dummies.drop(columns = ['registered', 'casual', 'cnt']) 
y = df_dummies[['registered']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_test =lr.predict(x_test)
y_pred_train =lr.predict(x_train)

def metricas(y_test, y_train, y_test_pred, y_train_pred):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    return df

results = metricas(y_test, y_train,y_pred_test, y_pred_train)
results

,MAE,MSE,RMSE,R2,set
0,439.834183,296735.441603,544.734285,0.862532,test
1,487.228377,447024.728917,668.599079,0.820694,train


    Parece que mejora algo más el R2 codificando con dummies.

- Estadarizamos sin categorizar.

In [51]:
df_estandarizado2 = df_mapa.copy()

In [22]:
df_estandarizado = df_dummies.copy()

In [24]:
st = StandardScaler()
x_tranformadas = pd.DataFrame(st.fit_transform(df_estandarizado))
x_tranformadas.columns = df_estandarizado.columns
df_estandarizado[x_tranformadas.columns] = x_tranformadas
df_estandarizado.head()

,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_autumn,season_spring,season_summer,season_winter
0,-1.0,-1.602745,-0.172102,1.497785,-1.469753,1.112127,-0.827613,-0.680818,1.252343,-0.387833,-0.755455,-1.927745,-1.820972,-0.588951,1.748015,-0.580514,-0.567859
1,-1.0,-1.602745,-0.172102,-1.495052,-1.469753,1.112127,-0.722069,-0.741507,0.480996,0.748899,-1.046996,-1.917480,-1.916078,-0.588951,1.748015,-0.580514,-0.567859
2,-1.0,-1.602745,-0.172102,-0.996246,0.680387,-0.724644,-1.635432,-1.750344,-1.338073,0.745931,-1.063031,-1.558846,-1.632827,-0.588951,1.748015,-0.580514,-0.567859
3,-1.0,-1.602745,-0.172102,-0.497440,0.680387,-0.724644,-1.615560,-1.610886,-0.261577,-0.389769,-1.080523,-1.414494,-1.522732,-0.588951,1.748015,-0.580514,-0.567859
4,-1.0,-1.602745,-0.172102,0.001367,0.680387,-0.724644,-1.468226,-1.505615,-1.340294,-0.046477,-1.118424,-1.373434,-1.503090,-0.588951,1.748015,-0.580514,-0.567859


In [52]:
st = StandardScaler()
x_tranformadas = pd.DataFrame(st.fit_transform(df_estandarizado2))
x_tranformadas.columns = df_estandarizado2.columns
df_estandarizado2[x_tranformadas.columns] = x_tranformadas
df_estandarizado2.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,-1.350819,-1.0,-1.602745,-0.172102,1.497785,-1.469753,1.112127,-0.827613,-0.680818,1.252343,-0.387833,-0.755455,-1.927745,-1.820972
1,-1.350819,-1.0,-1.602745,-0.172102,-1.495052,-1.469753,1.112127,-0.722069,-0.741507,0.480996,0.748899,-1.046996,-1.917480,-1.916078
2,-1.350819,-1.0,-1.602745,-0.172102,-0.996246,0.680387,-0.724644,-1.635432,-1.750344,-1.338073,0.745931,-1.063031,-1.558846,-1.632827
3,-1.350819,-1.0,-1.602745,-0.172102,-0.497440,0.680387,-0.724644,-1.615560,-1.610886,-0.261577,-0.389769,-1.080523,-1.414494,-1.522732
4,-1.350819,-1.0,-1.602745,-0.172102,0.001367,0.680387,-0.724644,-1.468226,-1.505615,-1.340294,-0.046477,-1.118424,-1.373434,-1.503090


In [26]:
x = df_estandarizado.drop(columns = ['registered', 'casual', 'cnt']) 
y = df_estandarizado[['registered']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_test =lr.predict(x_test)
y_pred_train =lr.predict(x_train)

def metricas(y_test, y_train, y_test_pred, y_train_pred):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    return df

results = metricas(y_test, y_train,y_pred_test, y_pred_train)
results

,MAE,MSE,RMSE,R2,set
0,0.282182,0.122138,0.349482,0.862532,test
1,0.312588,0.183998,0.428949,0.820694,train


In [53]:
x = df_estandarizado2.drop(columns = ['registered', 'casual', 'cnt']) 
y = df_estandarizado2[['registered']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_test =lr.predict(x_test)
y_pred_train =lr.predict(x_train)

def metricas(y_test, y_train, y_test_pred, y_train_pred):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    return df

results = metricas(y_test, y_train,y_pred_test, y_pred_train)
results

,MAE,MSE,RMSE,R2,set
0,0.287124,0.126517,0.355692,0.857603,test
1,0.328759,0.197329,0.444218,0.807702,train


    Obtenemos un R2 más alto que los anteriores y nuestras demás métricas son buenas. Hay un error de 0.3 y 0.4. Bastante bajos.

- Codificamos y volvemos a estandarizar

In [27]:
df_dummies['mnth'] = df_dummies['mnth'].astype('category')
df_dummies['yr'] = df_dummies['yr'].astype('category')
df_dummies['holiday'] = df_dummies['holiday'].astype('category')

In [28]:
df_dummies["weekday"] = df_dummies["weekday"].astype('category')
df_dummies["workingday"] = df_dummies["workingday"].astype('category')
df_dummies["weathersit"] = df_dummies["weathersit"].astype('category')

In [29]:
df_estand_cate = df_dummies.copy()

In [31]:
st = StandardScaler()
x_tranformadas = pd.DataFrame(st.fit_transform(df_estand_cate))
x_tranformadas.columns = df_estand_cate.columns
df_estandarizado[x_tranformadas.columns] = x_tranformadas
df_estandarizado.head()

,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_autumn,season_spring,season_summer,season_winter
0,-1.0,-1.602745,-0.172102,1.497785,-1.469753,1.112127,-0.827613,-0.680818,1.252343,-0.387833,-0.755455,-1.927745,-1.820972,-0.588951,1.748015,-0.580514,-0.567859
1,-1.0,-1.602745,-0.172102,-1.495052,-1.469753,1.112127,-0.722069,-0.741507,0.480996,0.748899,-1.046996,-1.917480,-1.916078,-0.588951,1.748015,-0.580514,-0.567859
2,-1.0,-1.602745,-0.172102,-0.996246,0.680387,-0.724644,-1.635432,-1.750344,-1.338073,0.745931,-1.063031,-1.558846,-1.632827,-0.588951,1.748015,-0.580514,-0.567859
3,-1.0,-1.602745,-0.172102,-0.497440,0.680387,-0.724644,-1.615560,-1.610886,-0.261577,-0.389769,-1.080523,-1.414494,-1.522732,-0.588951,1.748015,-0.580514,-0.567859
4,-1.0,-1.602745,-0.172102,0.001367,0.680387,-0.724644,-1.468226,-1.505615,-1.340294,-0.046477,-1.118424,-1.373434,-1.503090,-0.588951,1.748015,-0.580514,-0.567859


In [32]:
x = df_estand_cate.drop(columns = ['registered', 'casual', 'cnt']) 
y = df_estand_cate[['registered']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_test =lr.predict(x_test)
y_pred_train =lr.predict(x_train)

def metricas(y_test, y_train, y_test_pred, y_train_pred):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    return df

results = metricas(y_test, y_train,y_pred_test, y_pred_train)
results

,MAE,MSE,RMSE,R2,set
0,439.834183,296735.441603,544.734285,0.862532,test
1,487.228377,447024.728917,668.599079,0.820694,train


    Al categorizar algunas de nuestras variables la métrica vuelve a ser mucho peor, tendremos un margen de error bastante grande por lo que elegimos mejor no categorizar.

VIF

In [33]:
def get_VIF(dataframe, target):
        X = add_constant(dataframe.loc[:, dataframe.columns != target])
        seriesObject = pd.Series([variance_inflation_factor(X.values,i) for i in range(X.shape[1])] , index=X.columns,).sort_values(ascending= False)
        return seriesObject

In [34]:
get_VIF(df_estandarizado,'registered')

season_winter          inf
season_summer          inf
season_spring          inf
season_autumn          inf
temp             71.435232
atemp            66.821206
cnt               9.608703
casual            6.010899
mnth              3.438728
workingday        3.138069
yr                2.884768
hum               1.986696
weathersit        1.938942
windspeed         1.292986
holiday           1.101205
weekday           1.054259
const             1.000000
dtype: float64

    Observamos que temp, atemp, cnt y casual tienen una multicolinealidad por encima de 5, por lo tanto tienen una colinealida severa con los demás datos, pasaremos a eliminarlos poco a poco.

In [35]:
get_VIF(df_estandarizado.drop(columns = ['temp','atemp']),'registered')

season_autumn         inf
season_spring         inf
season_summer         inf
season_winter         inf
cnt              8.504283
casual           5.939928
mnth             3.434253
workingday       3.105009
yr               2.622877
weathersit       1.935529
hum              1.860949
windspeed        1.232649
holiday          1.092413
weekday          1.040641
const            1.000000
dtype: float64

In [36]:
get_VIF(df_estandarizado.drop(columns = ['temp','atemp', 'casual', 'cnt']),'registered')

season_autumn         inf
season_spring         inf
season_summer         inf
season_winter         inf
mnth             3.428152
hum              1.860145
weathersit       1.685377
windspeed        1.171519
holiday          1.082291
workingday       1.073285
weekday          1.017904
yr               1.016316
const            1.000000
dtype: float64

- Probamos a eliminar en nuestro modelo también las variables indicadas en el VIF.

In [41]:
x = df_estandarizado.drop(columns = ['registered', 'casual', 'cnt', 'atemp', 'temp']) 
y = df_estandarizado[['registered']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_test =lr.predict(x_test)
y_pred_train =lr.predict(x_train)

def metricas(y_test, y_train, y_test_pred, y_train_pred):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    return df

results = metricas(y_test, y_train,y_pred_test, y_pred_train)
results

,MAE,MSE,RMSE,R2,set
0,0.324826,0.176972,0.420680,0.800815,test
1,0.357250,0.225604,0.474977,0.780149,train


    Al eliminar temp y atemp el modelo empeora tanto en r2 como en margen de error, probemos a eliminar solo una, en este caso 'atemp' ya que en el MANOVA salía con un p-valor mucho mayor que 0.05 por lo que esta muy correlacionado con las demás variables(no es independiente).

In [43]:
x = df_estandarizado.drop(columns = ['registered', 'casual', 'cnt','atemp']) 
y = df_estandarizado[['registered']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_test =lr.predict(x_test)
y_pred_train =lr.predict(x_train)

def metricas(y_test, y_train, y_test_pred, y_train_pred):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    return df

results = metricas(y_test, y_train,y_pred_test, y_pred_train)
results

,MAE,MSE,RMSE,R2,set
0,0.283196,0.122636,0.350194,0.861971,test
1,0.312676,0.184002,0.428954,0.820690,train


    El modelo ha mejorado significativamente, asi que quitaremos atemp y dejaremos temp. Ahora pasamos a comprobar si nuestro modelo es correcto

### Conclusión

El mejor modelo es:
- Sin reemplazar outliers
- Codificando 'season' con dummies
- Sin categorizar
- Estandarizando con StandarScale

### Probamos nuestro modelo

In [58]:
df_estandarizado

,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_autumn,season_spring,season_summer,season_winter
0,-1.0,-1.602745,-0.172102,1.497785,-1.469753,1.112127,-0.827613,-0.680818,1.252343,-0.387833,-0.755455,-1.927745,-1.820972,-0.588951,1.748015,-0.580514,-0.567859
1,-1.0,-1.602745,-0.172102,-1.495052,-1.469753,1.112127,-0.722069,-0.741507,0.480996,0.748899,-1.046996,-1.917480,-1.916078,-0.588951,1.748015,-0.580514,-0.567859
2,-1.0,-1.602745,-0.172102,-0.996246,0.680387,-0.724644,-1.635432,-1.750344,-1.338073,0.745931,-1.063031,-1.558846,-1.632827,-0.588951,1.748015,-0.580514,-0.567859
3,-1.0,-1.602745,-0.172102,-0.497440,0.680387,-0.724644,-1.615560,-1.610886,-0.261577,-0.389769,-1.080523,-1.414494,-1.522732,-0.588951,1.748015,-0.580514,-0.567859
4,-1.0,-1.602745,-0.172102,0.001367,0.680387,-0.724644,-1.468226,-1.505615,-1.340294,-0.046477,-1.118424,-1.373434,-1.503090,-0.588951,1.748015,-0.580514,-0.567859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,1.0,1.587648,-0.172102,0.500173,0.680387,1.112127,-1.319509,-1.521749,0.177576,2.059845,-0.877902,-1.149528,-1.237414,-0.588951,1.748015,-0.580514,-0.567859
726,1.0,1.587648,-0.172102,0.998979,0.680387,1.112127,-1.324068,-1.347378,-0.264634,-0.452029,-0.299193,-0.774854,-0.730355,-0.588951,1.748015,-0.580514,-0.567859
727,1.0,1.587648,-0.172102,1.497785,-1.469753,1.112127,-1.324068,-1.425011,0.880424,-0.853182,-1.006180,-1.588999,-1.636962,-0.588951,1.748015,-0.580514,-0.567859
728,1.0,1.587648,-0.172102,-1.495052,-1.469753,-0.724644,-1.310404,-1.490698,-1.014341,2.067858,-0.707351,-1.428608,-1.401782,-0.588951,1.748015,-0.580514,-0.567859


In [74]:
X = df_estandarizado.drop(['atemp', 'casual', 'registered', 'cnt'], axis =1)
y = df_estandarizado[['registered']]

In [75]:
X_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [76]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [77]:
y_pred_test = lr.predict(x_test)
y_pred_train = lr.predict(X_train)

In [78]:
def metricas(y_test, y_train, y_test_pred, y_train_pred): 
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), 
                          metrics.mean_absolute_error(y_train, y_train_pred)], 
                  'MSE': [metrics.mean_squared_error(y_test, y_test_pred), 
                          metrics.mean_squared_error(y_train, y_train_pred)], 
                  'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), 
                           np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))], 
                  'R2': [metrics.r2_score(y_test, y_test_pred), 
                         metrics.r2_score(y_train, y_train_pred)], "set": ["test", "train"]} 
    df = pd.DataFrame(resultados) 
    return df 

In [79]:
modelito = {'yr':[1], 'mnth':[8], 'holiday':[1],'weekday':[4], 'workingday':[1], 'weathersit':[1], 'temp':[38], 'atemp':[45], 'hum':[30], 'windspeed':[10], 'season_summer': [1], 'season_winter': [0], 'season_autumn': [0], 'season_spring': [0]}

In [80]:
modelo = pd.DataFrame(modelito)

In [81]:
lr.predict(modelo)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 13 is different from 14)